In [12]:
from src.point2D import Point2D
from src.point3D import Point3D
from src.camera_model import Camera
from src.optimizetion import Optimizer
from src.initsolution import calc_init_camera
from src.plot import Plot
from src.distance import gps_to_enu

## Вычисление начальных значений

In [13]:
Line_Y = [[[786, 689], [281, 515]], [[1061, 516], [213, 340]], [[1008, 421], [375, 311]], [[355, 264], [70, 238]],
          [[362, 223], [73, 211]]]
Line_X = [[[300, 513], [555, 185]], [[835, 677], [927, 264]], [[674, 117], [740, 13]], [[833, 192], [842, 158]],
          [[954, 142], [954, 38]]]
camera = calc_init_camera('data/crossroads_pushkin_aksakov/crossroads_not_dist.jpg', [Line_X, Line_Y])

1080 1920


## Калибровка камеры

In [26]:
lines_calib = [
    [[831, 689, 7.71, 0, 0],
     [299, 520, 7.71, 20.7, 0]],
    [[896, 405, 18.9, 0, 0],
     [439, 322, 18.9, 20.7, 0]],
    [[927, 267, 31.9, 0, 0],
     [553, 189, 31.9, 20.7, 0]],
    [[890, 478, 15.09, 0, 0],
     [397, 379, 15.09, 20.7, 0]],
    [[300, 515, 7.71, 20.7, 0],
     [557, 180, 31.9, 20.7, 0]]
]

lines_prep = []
for line in lines_calib:
    start, end = line
    start2D, start3D = Point2D.from_homogeneous(start[0:2]), Point3D.from_homogeneous(start[2:6])
    end2D, end3D = Point2D.from_homogeneous(end[0:2]), Point3D.from_homogeneous(end[2:6])

    lines_prep.append([(start2D, start3D), (end2D, end3D)])

In [27]:
optimize = Optimizer(camera)

camera, info = optimize.optimize(lines_prep)

In [7]:
print(f'Углы поворота оптимизированные : {camera.get_R(angle_output=True)}')
print(f'Фокусное расстояние оптимизированное: {camera.get_f()}')
print(f'Вектор переноса оптимизированный: {camera.get_T()}')

Углы поворота оптимизированные : [ -99.58434695   37.91236625 -167.6947188 ]
Фокусное расстояние оптимизированное: 931.4576315372068
Вектор переноса оптимизированный: [ 0.          0.         31.72150652]


In [17]:
import ipywidgets as widgets
from ipywidgets import  interact
plot = Plot(camera)

def picking_params(f, alpha, beta, gamma, x, y, z):
    return plot.draw_tranform_line(LINE_PREP, out_jupyter=True, params=[f, alpha, beta, gamma, x, y ,z])

In [31]:
ref_lat, ref_lon = 54.723767, 55.933369

LINE_CALIB = [
                # [[54.723767, 55.933369, 779, 874], [54.723936, 55.933454, 600, 452]],
              # [[54.723767, 55.933369, 779, 874], [54.723714, 55.933668, 1399, 694]],
              # [[54.723714, 55.933668, 1399, 694], [54.723884, 55.933750, 1084, 344]],
              # [[54.723884, 55.933750, 1084, 344], [54.723936, 55.933454, 600, 452]],
              # [[54.723854, 55.933420, 679, 625], [54.723804, 55.933712, 1222, 481]],
              [[54.723735, 55.933514, 1133, 790], [54.723917, 55.933596, 815, 394]]]

LINE_CALIB_NEW = []
# Переводим координаты первой линии в ENU
for line in LINE_CALIB:
    (lat1, lon1, x1, y1), (lat2, lon2, x2, y2) = line
    e1, n1 = gps_to_enu(lat1, lon1, ref_lat, ref_lon)
    e2, n2 = gps_to_enu(lat2, lon2, ref_lat, ref_lon)

    LINE_CALIB_NEW.append([[x1, y1, float(e1), float(n1), 0], [x2, y2, float(e2), float(n2), 0]])

LINE_PREP = []
for line in LINE_CALIB_NEW:
    start, end = line
    start2D, start3D = Point2D(start[0:2]), Point3D(start[2:6])
    end2D, end3D = Point2D(end[0:2]), Point3D(end[2:6])

    LINE_PREP.append([(start2D, start3D), (end2D, end3D)])

    

In [32]:
interact(picking_params,  
         f=widgets.IntSlider(value=894, min=500, max=1200, step=0.1),  
         alpha=widgets.IntSlider(value=-138, min=-360, max=360, step=0.1),  
         beta=widgets.IntSlider(value=16, min=-360, max=360, step=0.1),  
         gamma=widgets.IntSlider(value=-186, min=-360, max=360, step=0.1),  
         x=widgets.IntSlider(value=6, min=0, max=40, step=0.1),  
         y=widgets.IntSlider(value=4, min=0, max=40, step=0.1),  
         z=widgets.IntSlider(value=25, min=0, max=40, step=0.1))

interactive(children=(IntSlider(value=894, description='f', max=1200, min=500, step=0), IntSlider(value=-138, …

<function __main__.picking_params(f, alpha, beta, gamma, x, y, z)>